In [6]:
import pandas as pd
import pingouin as pg
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import os

/home/predatt/giaald/.conda/envs/giacomo37/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/home/predatt/giaald/.conda/envs/giacomo37/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.0, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


## Second-level analysis

In [20]:
from utils import Options, loadmat
from configs import bids_dir
from mvpa.loading import get_correct_model

def load_labeled_contrasts(opt):
    """
    'opt' should contain sub, task, model
    """
    contr_dir = bids_dir+'derivatives/spm-preproc/derivatives/spm-stats/contrasts/'
    datamodel = get_correct_model(opt)
    data_dir = os.path.join(contr_dir, f'{opt.sub}/{opt.task}/model_{datamodel:g}/')
    
    SPM = loadmat(os.path.join(data_dir, 'SPM.mat'))
    regr_names = [n[6:-6] if '*bf(1)' in n else n[6:] for n in SPM['SPM']['xX']['name']]
    file_names = [os.path.join(data_dir, b.fname) for b in SPM['SPM']['Vbeta']]
    
    exclude = ['buttonpress', 'constant', 'tx', 'ty', 'tz', 'rx', 'ry', 'rz']
    
    file_names = [f for f, r in zip(file_names, regr_names) if r not in exclude]
    regr_names = [r for r in regr_names if r not in exclude]
    
    return regr_names, file_names


In [21]:
opt = Options(
    sub='sub-001',
    task='test',
    model=5
)
regr_names, file_names = load_labeled_contrasts(opt)

In [22]:
file_names

['/project/3018040.05/bids/derivatives/spm-preproc/derivatives/spm-stats/contrasts/sub-001/test/model_5/beta_0001.nii',
 '/project/3018040.05/bids/derivatives/spm-preproc/derivatives/spm-stats/contrasts/sub-001/test/model_5/beta_0002.nii',
 '/project/3018040.05/bids/derivatives/spm-preproc/derivatives/spm-stats/contrasts/sub-001/test/model_5/beta_0009.nii',
 '/project/3018040.05/bids/derivatives/spm-preproc/derivatives/spm-stats/contrasts/sub-001/test/model_5/beta_0010.nii',
 '/project/3018040.05/bids/derivatives/spm-preproc/derivatives/spm-stats/contrasts/sub-001/test/model_5/beta_0017.nii',
 '/project/3018040.05/bids/derivatives/spm-preproc/derivatives/spm-stats/contrasts/sub-001/test/model_5/beta_0018.nii',
 '/project/3018040.05/bids/derivatives/spm-preproc/derivatives/spm-stats/contrasts/sub-001/test/model_5/beta_0025.nii',
 '/project/3018040.05/bids/derivatives/spm-preproc/derivatives/spm-stats/contrasts/sub-001/test/model_5/beta_0026.nii',
 '/project/3018040.05/bids/derivatives/s

## ROI analysis

In [23]:
allfiles = glob('../../Univar_results/test_m5/*.csv')
allfiles = [f for f in allfiles if 'sub-006' not in f]

In [24]:
len(allfiles)

34

In [25]:
univar_df = []
for f in allfiles:
    univar_df.append(pd.read_csv(f))
univar_df = pd.concat(univar_df)

In [26]:
univar_df = univar_df.groupby(['condition', 'subject']).mean().reset_index()

In [27]:
univar_df.groupby(['condition'])['mean_beta'].mean()

condition
expected     -1.924122
unexpected   -1.847878
Name: mean_beta, dtype: float64

In [28]:
pg.ttest(univar_df[univar_df['condition']=='expected'].mean_beta,
         univar_df[univar_df['condition']=='unexpected'].mean_beta,
         paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.034888,33,two-sided,0.308246,"[-0.23, 0.07]",0.043261,0.301,0.056904
